In [1]:
from google.colab import drive
# Mount the Google Drive at mount
mount='/content/gdrive'
print("Colab: mounting Google drive on ", mount)

drive.mount(mount)

# Switch to the directory on the Google Drive that you want to use
import os
drive_root = mount + "/My Drive/NLP project/NLP_tr3"

# Create drive_root if it doesn't exist
create_drive_root = True
if create_drive_root:
  print("\nColab: making sure ", drive_root, " exists.")
  os.makedirs(drive_root, exist_ok=True)

# Change to the directory
print("\nColab: Changing directory to ", drive_root)
%cd $drive_root

Colab: mounting Google drive on  /content/gdrive
Mounted at /content/gdrive

Colab: making sure  /content/gdrive/My Drive/NLP project/NLP_tr3  exists.

Colab: Changing directory to  /content/gdrive/My Drive/NLP project/NLP_tr3
/content/gdrive/.shortcut-targets-by-id/1mteAAQwDYIOMp5NTHF20es2OzhXGCvMK/NLP_tr3


In [2]:
import time
import random
import numpy as np
import pickle as pkl
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

In [3]:
WITH_EXTRA_TRAIN = False
DATASET_PATH = drive_root + '/dataset'
CONSTANTS_PATH = drive_root + '/helpers/constants'

with open(CONSTANTS_PATH + '/ARABIC_LETTERS_LIST.pickle', 'rb') as file:
    ARABIC_LETTERS_LIST = pkl.load(file)
with open(CONSTANTS_PATH + '/DIACRITICS_LIST.pickle', 'rb') as file:
    DIACRITICS_LIST = pkl.load(file)
if not WITH_EXTRA_TRAIN:
    with open(CONSTANTS_PATH + '/RNN_SMALL_CHARACTERS_MAPPING.pickle', 'rb') as file:
        CHARACTERS_MAPPING = pkl.load(file)
else:
    with open(CONSTANTS_PATH + '/RNN_BIG_CHARACTERS_MAPPING.pickle', 'rb') as file:
        CHARACTERS_MAPPING = pkl.load(file)
with open(CONSTANTS_PATH + '/RNN_CLASSES_MAPPING.pickle', 'rb') as file:
    CLASSES_MAPPING = pkl.load(file)
with open(CONSTANTS_PATH + '/RNN_REV_CLASSES_MAPPING.pickle', 'rb') as file:
    REV_CLASSES_MAPPING = pkl.load(file)

In [4]:
train_raw = None
with open(DATASET_PATH + '/train.txt', 'r', encoding='utf-8') as file:
    train_raw = file.readlines()
if WITH_EXTRA_TRAIN:
    with open(DATASET_PATH + '/extra_train.txt', encoding='utf-8') as file:
        train_raw += file.readlines()
print('Training examples (raw):', len(train_raw))

val_raw = None
with open(DATASET_PATH + '/val.txt', encoding='utf-8') as file:
    val_raw = file.readlines()
print('Validation examples (raw):', len(val_raw))

Training examples (raw): 50000
Validation examples (raw): 2500


In [5]:
def remove_diacritics(data_raw):
    return data_raw.translate(str.maketrans('', '', ''.join(DIACRITICS_LIST)))

def to_one_hot(data, size):
    one_hot = list()
    for elem in data:
        cur = [0] * size
        cur[elem] = 1
        one_hot.append(cur)
    return one_hot

def split_data(data_raw):
    data_new = list()

    for line in data_raw:
        line = line.replace('.', '.\n')
        line = line.replace(',', ',\n')
        line = line.replace('،', '،\n')
        line = line.replace(':', ':\n')
        line = line.replace(';', ';\n')
        line = line.replace('؛', '؛\n')
        line = line.replace('(', '\n(')
        line = line.replace(')', ')\n')
        line = line.replace('[', '\n[')
        line = line.replace(']', ']\n')
        line = line.replace('{', '\n{')
        line = line.replace('}', '}\n')
        line = line.replace('«', '\n«')
        line = line.replace('»', '»\n')

        for sub_line in line.split('\n'):
            if len(remove_diacritics(sub_line).strip()) == 0:
                continue

            if len(remove_diacritics(sub_line).strip()) > 0 and len(remove_diacritics(sub_line).strip()) <= 500:
                data_new.append(sub_line.strip())
            else:
                sub_line = sub_line.split()
                tmp_line = ''
                for word in sub_line:
                    if len(remove_diacritics(tmp_line).strip()) + len(remove_diacritics(word).strip()) + 1 > 500:
                        if len(remove_diacritics(tmp_line).strip()) > 0:
                            data_new.append(tmp_line.strip())
                        tmp_line = word
                    else:
                        if tmp_line == '':
                            tmp_line = word
                        else:
                            tmp_line += ' '
                            tmp_line += word
                if len(remove_diacritics(tmp_line).strip()) > 0:
                    data_new.append(tmp_line.strip())

    return data_new

In [6]:
train_split = split_data(train_raw)
val_split = split_data(val_raw)
print('Training examples (split):', len(train_split))
print('Validation examples (split):', len(val_split))

Training examples (split): 299645
Validation examples (split): 15341


In [7]:
def map_data(data_raw):
    X = list()
    Y = list()

    for line in data_raw:
        x = [CHARACTERS_MAPPING['<SOS>']]
        y = [CLASSES_MAPPING['<SOS>']]

        for idx, char in enumerate(line):
            if char in DIACRITICS_LIST:
                continue

            x.append(CHARACTERS_MAPPING[char])

            if char not in ARABIC_LETTERS_LIST:
                y.append(CLASSES_MAPPING[''])
            else:
                char_diac = ''
                if idx + 1 < len(line) and line[idx + 1] in DIACRITICS_LIST:
                    char_diac = line[idx + 1]
                    if idx + 2 < len(line) and line[idx + 2] in DIACRITICS_LIST and char_diac + line[idx + 2] in CLASSES_MAPPING:
                        char_diac += line[idx + 2]
                    elif idx + 2 < len(line) and line[idx + 2] in DIACRITICS_LIST and line[idx + 2] + char_diac in CLASSES_MAPPING:
                        char_diac = line[idx + 2] + char_diac
                y.append(CLASSES_MAPPING[char_diac])

        assert(len(x) == len(y))

        x.append(CHARACTERS_MAPPING['<EOS>'])
        y.append(CLASSES_MAPPING['<EOS>'])

        y = to_one_hot(y, len(CLASSES_MAPPING))

        X.append(x)
        Y.append(y)

    X = np.asarray(X)
    Y = np.asarray(Y)

    return X, Y

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, SimpleRNN, Flatten,TimeDistributed, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import Sequence
from tensorflow.keras.initializers import glorot_normal
from tensorflow.keras.callbacks import ModelCheckpoint

In [9]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [10]:
def get_max(lines):
      X,Y = map_data(lines)
      X_max_seq_len = np.max([len(x) for x in X])
      return X_max_seq_len

max_length = get_max(train_split)
max_length

<ipython-input-7-5416c81cf8fa>:37: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.asarray(X)
<ipython-input-7-5416c81cf8fa>:38: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  Y = np.asarray(Y)


502

In [11]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(train_split)

In [ ]:
def get_max(lines):
      X,Y = map_data(lines)
      X_max_seq_len = np.max([len(x) for x in X])
      return X_max_seq_len

max_length = get_max(train_split)
max_length

In [ ]:
def create_model():

    # Calculate TF-IDF (assuming text_data is your text corpus):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(train_split)
    # Create input layer directly from TF-IDF matrix:
    inputs = Input(shape=(max_length, tfidf_matrix.shape[1]))  # Input shape is now TF-IDF features

    # Flatten TF-IDF matrix:

    rnn1 = SimpleRNN(units=256,
                    return_sequences=True,
                    kernel_initializer=glorot_normal(seed=961))(inputs)
    dropout1 = Dropout(0.5)(rnn1)
    rnn2 = SimpleRNN(units=256,
                    return_sequences=True,
                    kernel_initializer=glorot_normal(seed=961))(dropout1)
    dropout2 = Dropout(0.5)(rnn2)

    dense1 = TimeDistributed(Dense(units=512,
                                   activation='relu',
                                   kernel_initializer=glorot_normal(seed=961)))(dropout2)
    dense2 = TimeDistributed(Dense(units=512,
                                   activation='relu',
                                   kernel_initializer=glorot_normal(seed=961)))(dense1)

    output = TimeDistributed(Dense(units=len(CLASSES_MAPPING),
                                   activation='softmax',
                                   kernel_initializer=glorot_normal(seed=961)))(dense2)

    model = Model(inputs, output)


    #model.compile(loss='categorical_crossentropy', optimizer=Adam())
    model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

    return model

In [ ]:
model = create_model()
model.summary()

In [ ]:
class DataGenerator(Sequence):
    def __init__(self, lines, batch_size):
        self.lines = lines
        self.batch_size = batch_size
        self.vectorizer = vectorizer
    def __len__(self):
        return int(np.ceil(len(self.lines) / float(self.batch_size)))


    def __getitem__(self, idx):
        lines = self.lines[idx * self.batch_size:(idx + 1) * self.batch_size]
        X_batch, Y_batch = map_data(lines)

        # X_max_seq_len = np.max([len(x) for x in X_batch])
        # Y_max_seq_len = np.max([len(y) for y in Y_batch])

        X_max_seq_len = 502
        Y_max_seq_len = 502



        assert(X_max_seq_len == Y_max_seq_len)

        X = list()
        for x in X_batch:
            x = list(x)
            x.extend([CHARACTERS_MAPPING['<PAD>']] * (X_max_seq_len - len(x)))
            X.append(np.asarray(x))
        X_vec = vectorizer.transform(X)
        Y_tmp = list()
        for y in Y_batch:
            y_new = list(y)
            y_new.extend(to_one_hot([CLASSES_MAPPING['<PAD>']] * (Y_max_seq_len - len(y)), len(CLASSES_MAPPING)))
            Y_tmp.append(np.asarray(y_new))
        Y_batch = Y_tmp

        Y_batch = np.asarray(Y_batch)

        return np.asarray(X_vec), Y_batch

def fit_model(model, epochs, batch_size, train_split, val_split):
    random.shuffle(train_split)
    train_split = list(sorted(train_split, key=lambda line: len(remove_diacritics(line))))
    random.shuffle(val_split)
    val_split = list(sorted(val_split, key=lambda line: len(remove_diacritics(line))))

    checkpoint_path = 'checkpoints/epoch{epoch:02d}.ckpt'
    checkpoint_cb = ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, save_freq='epoch')

    training_generator = DataGenerator(train_split, batch_size)
    val_generator = DataGenerator(val_split, batch_size)

    model.fit(x=training_generator, validation_data=val_generator, epochs=epochs, callbacks=[checkpoint_cb])

In [ ]:
start_time = time.time()
fit_model(model, 5, 512, train_split, val_split)
end_time = time.time()
print('--- %s seconds ---' % round(end_time - start_time, 2))

In [ ]:
def predict(line, model):
    X, _ = map_data([line])
    predictions = model.predict(X).squeeze()
    predictions = predictions[1:]

    output = ''
    for char, prediction in zip(remove_diacritics(line), predictions):
        output += char

        if char not in ARABIC_LETTERS_LIST:
            continue

        if '<' in REV_CLASSES_MAPPING[np.argmax(prediction)]:
            continue

        output += REV_CLASSES_MAPPING[np.argmax(prediction)]

    return output